# Imports

In [1]:
import requests
import json
import numpy as np
import pandas as pd
import os

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.plotting import figure, show

# Definitions

In [11]:
cwd = os.getcwd()
leagueID = "1041760"
fpl_league_url = (
    f"https://fantasy.premierleague.com/api/leagues-classic/{leagueID}/standings/"
)
transfers_url_template = (
    "https://fantasy.premierleague.com/api/entry/{manager_id}/transfers/"
)

# Testing

## Main league df

In [12]:
respose = requests.get(fpl_league_url)
if respose.status_code == 200:
    print("The API request was successful.")
    fpl_league_respose_json = respose.json()
else:
    print(f"Error: The API request failed with status code {respose.status_code}.")

The API request was successful.


In [13]:
league_df = pd.DataFrame(fpl_league_respose_json["standings"]["results"])
league_df

,id,event_total,player_name,rank,last_rank,rank_sort,total,entry,entry_name
0,35832424,58,James Parfitt,1,1,1,1313,4651100,Where you been?
1,58581266,56,Seb Winn,2,2,2,1280,7052472,Gangsters Allardyce
2,35939248,50,Alex Winn,3,3,3,1270,3794145,Al-Winn
3,35832439,36,Jacob Pickles,4,4,4,1217,3148659,Gobblebox
4,57395169,29,Mason Gardener,5,5,5,1210,6963106,nice to Michu
5,35832433,64,Ben Melhuish,6,7,6,1206,4075412,Melshys Mediocres
6,35832437,52,Dan Pratlett,7,6,7,1203,3194123,Onana what’s my name
7,35879230,62,Joseph Parsons,8,9,8,1194,4656770,AFC Great Name UTD
8,56846765,56,Richard Collins,9,8,9,1193,6911033,Eze-er said than Son
9,48472859,60,Ben Wilson,10,11,10,1114,6083206,Kloppenheimer


## Open player data and make ID map

In [14]:
players_df = pd.read_csv(cwd + "/../data/app/players_raw.csv")
players_df.loc[:, ["id", "web_name"]]
id_name_dict = pd.Series(
    players_df["web_name"].values, index=players_df["id"]
).to_dict()
id_name_dict

{1: 'Balogun',
 2: 'Cédric',
 3: 'M.Elneny',
 4: 'Fábio Vieira',
 5: 'Gabriel',
 6: 'Havertz',
 8: 'G.Jesus',
 9: 'Jorginho',
 10: 'Kiwior',
 11: 'Marquinhos',
 12: 'Martinelli',
 13: 'Nketiah',
 14: 'Ødegaard',
 15: 'Thomas',
 16: 'Pepe',
 17: 'Ramsdale',
 18: 'Rúnarsson',
 19: 'Saka',
 20: 'Saliba',
 22: 'Smith Rowe',
 24: 'Tierney',
 25: 'Tomiyasu',
 26: 'Trossard',
 29: 'White',
 30: 'Xhaka',
 31: 'Zinchenko',
 113: 'Raya',
 540: 'Rice',
 578: 'Nelson',
 585: 'J.Timber',
 646: 'Hein',
 735: 'Sagoe',
 736: 'Walters',
 737: 'Ibrahim',
 764: 'Lewis-Skelly',
 772: 'Nwaneri',
 32: 'Alex Moreno',
 34: 'Bailey',
 35: 'Buendia',
 36: 'Cash',
 37: 'Chambers',
 38: 'Coutinho',
 39: 'Davis',
 40: 'Dendoncker',
 41: 'Diego Carlos',
 42: 'Digne',
 43: 'Douglas Luiz',
 44: 'Duran',
 45: 'Hause',
 46: 'Iroegbunam',
 47: 'Kamara',
 48: 'Konsa',
 49: 'Martinez',
 50: 'McGinn',
 51: 'Mings',
 53: 'Olsen',
 54: 'Philogene',
 55: 'J.Ramsey',
 56: 'Sanson',
 57: 'Sinisalo',
 58: 'Tielemans',
 59: 'Bert

## Do the same for manager_id

In [15]:
manager_id_name_dict = pd.Series(
    league_df["player_name"].values, index=league_df["entry"]
).to_dict()
manager_id_name_dict

{4651100: 'James Parfitt',
 7052472: 'Seb Winn',
 3794145: 'Alex Winn',
 3148659: 'Jacob Pickles',
 6963106: 'Mason Gardener',
 4075412: 'Ben Melhuish',
 3194123: 'Dan Pratlett',
 4656770: 'Joseph Parsons',
 6911033: 'Richard Collins',
 6083206: 'Ben Wilson',
 2709514: 'Ben Anderson',
 4371621: 'Luke Counsell'}

## Get tranfers df

In [29]:
transfers_dfs_list = []
for manager_id in league_df["entry"].values:
    respose = requests.get(transfers_url_template.format(manager_id=manager_id))
    transfers_respose_json = respose.json()
    if not transfers_respose_json:
        continue
    transfers_df = pd.DataFrame(transfers_respose_json)
    transfers_df["element_in"] = transfers_df["element_in"].map(id_name_dict)
    transfers_df["element_in_cost"] = transfers_df["element_in_cost"] * 1e5
    transfers_df["element_out"] = transfers_df["element_out"].map(id_name_dict)
    transfers_df["element_out_cost"] = transfers_df["element_out_cost"] * 1e5
    transfers_df["manager_id"] = manager_id
    transfers_df["Manager"] = manager_id_name_dict[manager_id]
    transfers_df["time"] = pd.to_datetime(transfers_df["time"]).dt.round("1s")
    transfers_dfs_list.append(transfers_df)
all_managers_transfers_df = pd.concat(transfers_dfs_list)
all_managers_transfers_df.dtypes

element_in                       object
element_in_cost                 float64
element_out                      object
element_out_cost                float64
entry                             int64
event                             int64
time                datetime64[ns, UTC]
manager_id                        int64
Manager                          object
dtype: object

In [33]:
all_managers_transfers_df

,element_in,element_in_cost,element_out,element_out_cost,entry,event,time,manager_id,Manager
0,Diogo J.,7800000.0,Salah,12800000.0,4651100,21,2024-01-12 13:46:35+00:00,4651100,James Parfitt
1,Foden,7600000.0,Son,9800000.0,4651100,20,2023-12-30 00:15:34+00:00,4651100,James Parfitt
2,Alexander-Arnold,8300000.0,Tsimikas,4700000.0,4651100,19,2023-12-24 23:25:16+00:00,4651100,James Parfitt
3,Richarlison,6700000.0,Gordon,5800000.0,4651100,18,2023-12-21 17:09:40+00:00,4651100,James Parfitt
4,Son,9700000.0,Kudus,6600000.0,4651100,17,2023-12-15 17:28:35+00:00,4651100,James Parfitt
...,...,...,...,...,...,...,...,...,...
48,J.Alvarez,6500000.0,Darwin,7400000.0,2709514,3,2023-08-20 09:45:24+00:00,2709514,Ben Anderson
49,Wan-Bissaka,4500000.0,Maguire,4400000.0,2709514,3,2023-08-20 09:45:24+00:00,2709514,Ben Anderson
50,Chilwell,5700000.0,R.Lewis,4500000.0,2709514,3,2023-08-20 09:45:24+00:00,2709514,Ben Anderson
51,Al-Dakhil,4000000.0,J.Timber,5000000.0,2709514,2,2023-08-12 13:12:10+00:00,2709514,Ben Anderson
